In [65]:
import nltk
import pandas as pd
import numpy as np
from nltk.corpus import stopwords, wordnet
import itertools
import matplotlib.pyplot as plt
from sklearn import manifold
from nltk import pos_tag, word_tokenize
from nltk.probability import FreqDist
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import string


In [9]:
df = pd.read_table('/Users/brettscroggins/Downloads/airline.csv', sep=',')
airlines = df[['airline_name', 'content']]
airlines[:5]

,airline_name,content
0,adria-airways,Outbound flight FRA/PRN A319. 2 hours 10 min f...
1,adria-airways,Two short hops ZRH-LJU and LJU-VIE. Very fast ...
2,adria-airways,Flew Zurich-Ljubljana on JP365 newish CRJ900. ...
3,adria-airways,Adria serves this 100 min flight from Ljubljan...
4,adria-airways,WAW-SKJ Economy. No free snacks or drinks on t...


In [10]:
# airlines['airline_name'].value_counts()


# mapping that function to forum df
stop = stopwords.words('english')
airlines['content_clean'] = airlines['content'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
airlines[:5]

/Users/brettscroggins/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:6: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  
/Users/brettscroggins/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,airline_name,content,content_clean
0,adria-airways,Outbound flight FRA/PRN A319. 2 hours 10 min f...,Outbound flight FRA/PRN A319. 2 hours 10 min f...
1,adria-airways,Two short hops ZRH-LJU and LJU-VIE. Very fast ...,Two short hops ZRH-LJU LJU-VIE. Very fast CRJ ...
2,adria-airways,Flew Zurich-Ljubljana on JP365 newish CRJ900. ...,Flew Zurich-Ljubljana JP365 newish CRJ900. Fli...
3,adria-airways,Adria serves this 100 min flight from Ljubljan...,Adria serves 100 min flight Ljubljana Amsterda...
4,adria-airways,WAW-SKJ Economy. No free snacks or drinks on t...,WAW-SKJ Economy. No free snacks drinks Star Al...


In [12]:
temp = airlines[:5]

In [13]:
temp['sent_score'] = ''

/Users/brettscroggins/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [14]:
sid = SentimentIntensityAnalyzer()

for i in range(len(temp)):
    
    line = temp['content_clean'][i]
    
    ss = sid.polarity_scores(line)

    temp['sent_score'][i] = ss

In [15]:
temp['sent_score'][0]

{'compound': 0.5859, 'neg': 0.066, 'neu': 0.783, 'pos': 0.151}

In [14]:
temp['content_clean'][0]

"Outbound flight FRA/PRN A319. 2 hours 10 min flight. I thought drinks/snacks sale sandwich soft drinks served complimentary. Inbound flights SKP/LJU/FRA CRJ900. 1 hour 30 min flight. Skyshop menu seat pocket drinks/snacks sale. All flight crews friendly. Security check Ljubljana airport transit passengers chaos however it's possible go gate within 30min."

In [78]:
line = temp['content_clean'][0]
line = line.translate(None, string.punctuation)
line = line.split(' ')

In [90]:
words = ['passengers', 'crews']
dictionary = {}

for word in words:
    for i in range(len(line)):
        if word == line[i]:
            temp_list = line[i-2] + ' ' + line[i-1] + ' ' + line[i+1] + ' ' + line[i+2]
            x = {word:temp_list}
            dictionary.update(x)

dictionary

df = pd.DataFrame()
df['word'] = dictionary.keys()
df['string'] = dictionary.values()
# df['sent_score'] = ''
df['compound']=''
df

,word,string,compound
0,passengers,airport transit chaos however,
1,crews,All flight friendly Security,


In [91]:
for i in range(len(df)):
    
    temp_line = df['string'][i]
    
    ss = sid.polarity_scores(temp_line)

    df['compound'][i] = ss['compound']
    
#     df['compound'][i] = df['sent_score'][i]['compound']
    
df

,word,string,compound
0,passengers,airport transit chaos however,-0.5719
1,crews,All flight friendly Security,0.6808
